In [6]:
chan_index = int(input('Enter channel index to analyze:'))
# print('Enter Open Ephys data directory path:')
# source_path = raw_input()

Enter channel index to analyze:1


In [7]:
source_path = '../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/'

In [8]:
import time
t0 = time.time()
import OpenEphys
from kaveh.toolbox import common_avg_ref, butter_bandpass_filter
import Kwik
from matplotlib import pyplot as plt
import numpy as np
import os
import re
import scipy.signal

In [9]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [10]:
f_names = []
for root, dirnames, filenames in os.walk(source_path):
    for filename in filenames:
        f_regex = re.compile(r".*CH(\d|\d\d)\.continuous$")
        if f_regex.match(filename):
            f_names = f_names + [os.path.join(root, filename)]
f_names = np.array(f_names)
chans = [int(f.split('.')[-2].split('_')[-1][2:]) for f in f_names]
f_names = f_names[np.argsort(chans)] # now sorted by channel number

In [11]:
chan_data = []
for contact in range(7):
    contact_reps = []
    for i in range(contact*4, contact*4 + 4):
        print('Reading {}...'.format(f_names[i]))
        file_content = OpenEphys.load(f_names[i])
        Fs = float(file_content['header']['sampleRate'])
        signal_filtered = butter_bandpass_filter(file_content['data'], 200, 10000, Fs, order=2 )
        contact_reps.append(signal_filtered)
    contact_reps = np.array(contact_reps)
    chan_data.append(np.mean(contact_reps, axis=0))
    print('-----------------------------------------------')
chan_data = np.array(chan_data)
contact_reps = None

Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH1.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH2.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH3.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH4.continuous...
Loading continuous data...
-----------------------------------------------
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH5.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH6.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH7.continuous...
Loading continuous data...
Reading ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/100_CH8.continuous...
Loading continuous data...
-----------------------------------------------
Reading ../data/

In [12]:
common_avg_ref(chan_data)
chan_data = scipy.signal.detrend(chan_data)

chan_filtered_fname = os.path.join(source_path, 'channels_filtered.npy')
print('Saving filtered data to {}'.format(chan_filtered_fname))
np.save(chan_filtered_fname, chan_data)

Saving filtered data to ../data/OpenEphys_data/2019-09-09/2019-09-09_17-17-19/channels_filtered.npy


In [ ]:
# chan_filtered_fname = os.path.join(source_path, 'channels_filtered.npy')
# chan_data_raw = np.load(chan_filtered_fname)
# Fs = 30000.0

### Plot channel voltage

In [ ]:
# plot channel
# plt.figure(figsize=(16,5))
# plt.plot(chan_data[chan_index, :])
# plt.title('Channel {}. Signal length = {}s'.format(chan_index, chan_data.shape[1]/Fs))

In [13]:
from cssorter.spikesorter import ComplexSpikeSorter
dt = 1.0/Fs
css = ComplexSpikeSorter(chan_data[chan_index, :], dt)
css.num_gmm_components = 3
css.cs_num_gmm_components = 6
css.pre_window = 0.0002
css.post_window = 0.003
css.run(use_filtered=False, remove_overlap=False, spike_detection_dir = 'min', align_spikes_to='min')

Pre-process...
Using minibatch spike detection, batch size = 50s, using filter (derivative) is False
Using spikes min for detection. Aligning them to their min. 
Spike detection time = 8.739030599594116
Using all spikes for alignment (no overlap removal)
Align spikes time = 0.050180673599243164
Clustering complex spikes...
Splitting detected spikes for CS clustering ...
CS spike detection time = 1.2418863773345947
Merging overlapping CS waveforms
CS post process time = 0.0013968944549560547


In [ ]:
num_clusters = 12
pre_time = 0.0005
post_time = 0.005
clusters, cluster_labels = css.cluster_detected_cs(num_clusters=num_clusters, pre_time=pre_time, post_time=post_time)
ss_indices = css.get_ss_indices()

In [ ]:
ss_indices.shape

In [ ]:
import random
pre_index = int(np.round(pre_time/css.dt))
post_index = int(np.round(post_time/css.dt))
aligned_ss = np.array([css.voltage[i - pre_index : i + post_index] for i in ss_indices[1:-2]])
mean_ss = np.mean(aligned_ss[random.sample(range(0, aligned_ss.shape[0]), css.cs_indices.size), ], axis=0)

In [ ]:
t1 = time.time()
print('Total run time = {} s'.format(t1-t0))

In [ ]:
# plot cluster means
colors = plt.cm.nipy_spectral(np.linspace(0,1,num_clusters))
legend_labels = []
for cn in np.arange(num_clusters):
    legend_labels.append('c{}({}) '.format(cn, clusters[cn].shape[0]))

import gc
gc.collect()
plt.figure(figsize=(8,5))
clust_means = []
paxes = []
for cn in np.arange(num_clusters):
    clust_means.append(np.mean(clusters[cn], axis=0))
    ax = plt.plot(clust_means[-1], color = colors[cn], label = legend_labels[cn])
    paxes.append(ax)
plt.plot(mean_ss, '--', label = 'Mean SS({})'.format(aligned_ss.shape[0]))
plt.legend() 

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(clust_means)
cs_clust = np.argmax(np.sum(np.abs(kmeans.cluster_centers_), axis=1))
clusters_to_pick = np.where(kmeans.labels_ == cs_clust)[0]
print('Detected CS clusters: {}'.format(clusters_to_pick))

In [ ]:
ans = input("Enter CS clusters (comma separated; example: 5,3,1). Type a character to accept the detected cluster: ")
ans_regex = re.compile(r'\d+(?:,\d+)?')
if ans_regex.match(ans):
    clusters_to_pick = [int(c) for c in ans.split(',')]
    

In [20]:
cs_indices_to_pick = []
for cti in clusters_to_pick:
    cs_indices_to_pick = np.union1d(cs_indices_to_pick, css.cs_indices[np.where(cluster_labels == cti)])
cs_indices = cs_indices_to_pick.astype(np.int64)
print('Number of detected CS: {}'.format(cs_indices.shape))

Number of detected CS: (1,)


In [ ]:
# plot channel
f, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 1]}, figsize=(20,8), sharex=True)

# prange = [css.signal_size*css.dt - 100, css.signal_size*css.dt] #s
prange = [0,chan_data.shape[1]/Fs] #s
idx_voltage = [int(pr*Fs) for pr in prange]
ax0.plot(np.arange(prange[0],prange[1],1/Fs), chan_data[chan_index, idx_voltage[0]: idx_voltage[1]], alpha=0.5, color='k')
# ax1.eventplot(css.cs_indices[np.where(np.logical_and(css.cs_indices < idx_voltage[1] , css.cs_indices >= idx_voltage[0]))]*css.dt, linelengths=50, lineoffsets=0, color='g')
ax1.eventplot(cs_indices[np.where(np.logical_and(cs_indices < idx_voltage[1] , cs_indices >= idx_voltage[0]))]*css.dt, linelengths=50, lineoffsets=50, color='r')
# ax1.eventplot(css.spike_indices[np.where(np.logical_and(css.spike_indices < idx_voltage[1] , css.spike_indices >= idx_voltage[0]))]*css.dt, linelengths=50, lineoffsets=100, color='m')
# plt.plot(chan_data[chan_index,::10])
plt.title('Channel {}. Signal length = {}s'.format(chan_index, chan_data.shape[1]/Fs))

In [ ]:
window = 25

SS = np.zeros(np.int64(css.signal_size/(Fs/1000.0)))
SS[np.int64(ss_indices/(Fs/1000.0))] = 1
CS = np.zeros(np.int64(css.signal_size/(Fs/1000.0)))
CS[np.int64(cs_indices/(Fs/1000.0))] = 1



xcor_vect = np.zeros((2*window + 1))
first_cs_acor = np.argmax(cs_indices > window*(Fs/1000.0))
last_cs_acor = np.argmax(cs_indices[::-1] < ss_indices[-1] - window*30) + 1
for cti in np.int64(cs_indices[first_cs_acor:-1]/(Fs/1000.0)):
    xcor_vect = xcor_vect + SS[cti-window : cti + window + 1]

xcor_vect = xcor_vect / (cs_indices.size - (first_cs_acor + last_cs_acor))
    
acor_vect = np.zeros((2*window + 1))
first_ss_acor = np.argmax(ss_indices > window*(Fs/1000.0))
last_ss_acor = np.argmax(ss_indices[::-1] < ss_indices[-1] - window*(Fs/1000.0)) + 1
for sti in np.int64(ss_indices[first_ss_acor:-last_ss_acor]/(Fs/1000.0)):
    acor_vect = acor_vect + SS[sti-window : sti + window + 1]

acor_vect[window] = 0

acor_vect = acor_vect / (ss_indices.size - (first_ss_acor + last_ss_acor))

plt.figure()
plt.plot(xcor_vect, 'r', alpha=0.35)
plt.yticks(color='r')
plt.fill_between(np.arange(0,2*window + 1), xcor_vect, color='r', alpha=0.25)
plt.ylabel('P(SS(t) | CS(0)) Count', color = 'r')
plt.xlabel('Time (ms)')

plt.twinx()
plt.plot(acor_vect, 'k', alpha=0.70)
plt.yticks(color='k')
plt.ylabel('P(SS(t) | SS(0)) Count', color = 'k')
# plt.fill_between(acor_ret[0], 0, acor_ret[1] , color='g', alpha=0.25)
plt.title('Conditional Probabilities')

In [53]:
import numpy as np
from sklearn.mixture import GaussianMixture
num_clusters = 8
pre_time = 0.0005
post_time = 0.005
align_to = 'min'
pre_index = int(np.round(pre_time/css.dt))
post_index = int(np.round(post_time/css.dt))
delta = 60 # s
cs_indices_all = []
aligned_cs = []
for ti in np.arange(0, css.signal_size, delta*Fs ):
    print(ti)
    if align_to is 'min':
        aligned_cs = np.array([css.voltage[i - pre_index : i + post_index] for i in css.cs_indices if i < ti + delta*Fs and i > ti])
    elif align_to is 'max':
        aligned_cs = []
        for csi in css.cs_indices[np.where(np.logical_and(cs_indices < ti + delta*Fs, cs_indices>ti))]:
            align_point = np.argmax(css.voltage[csi - pre_index : csi + post_index]) + csi-pre_index
            aligned_cs.append(css.voltage[align_point - pre_index : align_point + post_index])
        aligned_cs = np.array(aligned_cs)
    else:
        raise ValueError("align_to should be min or max")

    gmm = GaussianMixture(num_clusters, covariance_type = 'full').fit(aligned_cs)
    cluster_labels = gmm.predict(aligned_cs)
    clusters = []

    for cn in np.arange(num_clusters):
        clusters.append(aligned_cs[np.where(cluster_labels == cn)])

    clust_means = []
    for cn in np.arange(num_clusters):
        clust_means.append(np.mean(clusters[cn], axis=0))

    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=2, random_state=0).fit(clust_means)
    cs_clust = np.argmax(np.sum(np.abs(kmeans.cluster_centers_), axis=1))
    clusters_to_pick = np.where(kmeans.labels_ == cs_clust)[0]

    cs_indices_to_pick = []
    for cti in clusters_to_pick:
        cs_indices_to_pick = np.union1d(cs_indices_to_pick, css.cs_indices[np.where(cluster_labels == cti)])
    cs_indices_all.append(cs_indices_to_pick.astype(np.int64))
cs_indices_all = np.hstack(cs_indices_all)
print(cs_indices_all.shape)

0.0
1800000.0
3600000.0
5400000.0
(176,)


In [38]:
np.hstack(cs_indices_all)

(110,)

## Saving detected CS and SS:

In [ ]:
CS_csv_filename = os.path.join(source_path, 'channel_{}.CS.csv'.format(chan_index))
SS_csv_filename = os.path.join(source_path, 'channel_{}.SS.csv'.format(chan_index))

import csv
with open(CS_csv_filename, 'w+') as f:
    print('writing {} ... '.format(CS_csv_filename))
    f.seek(0)
    csvwriter = csv.writer(f, delimiter = ',')
    csvwriter.writerows(cs_indices.reshape(-1,1))
    
with open(SS_csv_filename, 'w+') as f:
    print('writing {} ... '.format(SS_csv_filename))
    f.seek(0)
    csvwriter = csv.writer(f, delimiter = ',')
    csvwriter.writerows(ss_indices.reshape(-1,1))